In [ ]:
# change to code to  save the storage space .Earlier it used to calculate frames for all the video and then after storing it .It used to give all the frame for each video to get features.npy for each video. frame for 400 video of 2.1GB tool 76GB of space .hence it will not be  feasible use for larger dataset of 1000 or 10k video

# hence instead of doing ela for all video and giving input to CNN we Instead of processing multiple ELA images, we now only process the 'error_level_plot.png' for each video folder.We extract features from this single image and save them as 'features.npy'.


In [27]:
import os
import cv2
import numpy as np
from PIL import Image, ImageChops, ImageEnhance
import matplotlib.pyplot as plt

def resize_frame(frame, scale_factor=0.5):
    height, width = frame.shape[:2]
    new_size = (int(width * scale_factor), int(height * scale_factor))
    resized_frame = cv2.resize(frame, new_size, interpolation=cv2.INTER_LANCZOS4)
    return resized_frame

def apply_ela(image_path, scale_factor=0.5):
    original = Image.open(image_path)
    original = original.convert("RGB")

    resized = original.resize((int(original.width * scale_factor), int(original.height * scale_factor)), Image.Resampling.LANCZOS)

    original_np = np.array(original)
    resized_np = np.array(resized)

    error_level = ImageChops.difference(Image.fromarray(original_np), Image.fromarray(resized_np))

    error_level_np = np.array(error_level)

    mean_error = np.mean(error_level_np)

    return mean_error

def process_videos(video_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    video_files = [os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]

    for video_file in video_files:
        video_name = os.path.splitext(os.path.basename(video_file))[0]
        video_output_folder = os.path.join(output_folder, video_name)
        if not os.path.exists(video_output_folder):
            os.makedirs(video_output_folder)

        cap = cv2.VideoCapture(video_file)

        if not cap.isOpened():
            print(f"Error: Could not open video file {video_file}")
            continue

        frame_number = 0
        mean_errors = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            resized_frame = resize_frame(frame)

            frame_filename = os.path.join(video_output_folder, f"frame_{frame_number}.png")
            cv2.imwrite(frame_filename, resized_frame)

            mean_error = apply_ela(frame_filename)
            mean_errors.append(mean_error)

            os.remove(frame_filename)  # Delete the frame image after processing

            frame_number += 1

        cap.release()
        print(f"Processing complete for {video_file}")

        plt.figure(figsize=(10, 5))
        plt.plot(mean_errors, label=f'Error Level - {video_name}')
        plt.xlabel('Frame Number')
        plt.ylabel('Mean Error Level')
        plt.title(f'Mean Error Level Across Frames for {video_name}')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(video_output_folder, 'error_level_plot.png'))
        plt.close()

        # Save mean_errors as a numpy array
        np.save(os.path.join(video_output_folder, 'mean_errors.npy'), np.array(mean_errors))

# Example usage
video_folder = "train_sample_videos"
output_folder = "output_frames"

process_videos(video_folder, output_folder)

Processing complete for train_sample_videos/cdaxixbosp.mp4
Processing complete for train_sample_videos/btiysiskpf.mp4
Processing complete for train_sample_videos/clihsshdkq.mp4
Processing complete for train_sample_videos/alvgwypubw.mp4


KeyboardInterrupt: 

In [4]:
# modified above code to skip the file which has already been processed

import os
import cv2
import numpy as np
from PIL import Image, ImageChops, ImageEnhance
import matplotlib.pyplot as plt

def resize_frame(frame, scale_factor=0.5):
    height, width = frame.shape[:2]
    new_size = (int(width * scale_factor), int(height * scale_factor))
    resized_frame = cv2.resize(frame, new_size, interpolation=cv2.INTER_LANCZOS4)
    return resized_frame

def apply_ela(image_path, scale_factor=0.5):
    original = Image.open(image_path)
    original = original.convert("RGB")#L FOR TRANSLATING COLOR IMAGE TO GREYSCALE

    resized = original.resize((int(original.width * scale_factor), int(original.height * scale_factor)), Image.Resampling.LANCZOS)

    original_np = np.array(original)
    resized_np = np.array(resized)

    error_level = ImageChops.difference(Image.fromarray(original_np), Image.fromarray(resized_np))

    error_level_np = np.array(error_level)

    mean_error = np.mean(error_level_np)

    return mean_error

def process_videos(video_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    video_files = [os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]

    for video_file in video_files:
        video_name = os.path.splitext(os.path.basename(video_file))[0]
        video_output_folder = os.path.join(output_folder, video_name)

        # Skip processing if mean_errors.npy already exists
        if os.path.exists(os.path.join(video_output_folder, 'mean_errors.npy')):
            print(f"Skipping {video_file} as it has already been processed.")
            continue

        if not os.path.exists(video_output_folder):
            os.makedirs(video_output_folder)

        cap = cv2.VideoCapture(video_file)

        if not cap.isOpened():
            print(f"Error: Could not open video file {video_file}")
            continue

        frame_number = 0
        mean_errors = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            resized_frame = resize_frame(frame)

            frame_filename = os.path.join(video_output_folder, f"frame_{frame_number}.png")
            cv2.imwrite(frame_filename, resized_frame)

            mean_error = apply_ela(frame_filename)
            mean_errors.append(mean_error)

            os.remove(frame_filename)  # Delete the frame image after processing

            frame_number += 1

        cap.release()
        print(f"Processing complete for {video_file}")

        plt.figure(figsize=(10, 5))
        plt.plot(mean_errors, label=f'Error Level - {video_name}')
        plt.xlabel('Frame Number')
        plt.ylabel('Mean Error Level')
        plt.title(f'Mean Error Level Across Frames for {video_name}')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(video_output_folder, 'error_level_plot.png'))
        plt.close()

        # Save mean_errors as a numpy array
        np.save(os.path.join(video_output_folder, 'mean_errors.npy'), np.array(mean_errors))

# Example usage
video_folder = "train_sample_videos"
output_folder = "output_frames"

process_videos(video_folder, output_folder)


Skipping train_sample_videos/cdaxixbosp.mp4 as it has already been processed.
Skipping train_sample_videos/btiysiskpf.mp4 as it has already been processed.
Skipping train_sample_videos/clihsshdkq.mp4 as it has already been processed.
Skipping train_sample_videos/alvgwypubw.mp4 as it has already been processed.
Skipping train_sample_videos/eqvuznuwsa.mp4 as it has already been processed.
Skipping train_sample_videos/eudeqjhdfd.mp4 as it has already been processed.
Skipping train_sample_videos/eeyhxisdfh.mp4 as it has already been processed.
Skipping train_sample_videos/cizlkenljw.mp4 as it has already been processed.
Skipping train_sample_videos/bndybcqhfr.mp4 as it has already been processed.
Skipping train_sample_videos/cuzrgrbvil.mp4 as it has already been processed.
Skipping train_sample_videos/atyntldecu.mp4 as it has already been processed.
Skipping train_sample_videos/bggsurpgpr.mp4 as it has already been processed.
Skipping train_sample_videos/eckvhdusax.mp4 as it has already be

In [28]:
# USE PRETRAINED MODEL RESNET18 TO EXTRACT FEATURE
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

model = models.resnet18(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])
model.eval()

def extract_features(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        image_tensor = transform(image).unsqueeze(0)

        print(f"Processing image: {image_path}")
        print(f"Image tensor shape: {image_tensor.shape}")

        with torch.no_grad():
            features = model(image_tensor)
            print(f"Extracted features shape: {features.shape}")

        features = features.squeeze().numpy()
        print(f"Squeezed features shape: {features.shape}")
        return features
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

def process_video_folders(base_folder):
    for video_folder in os.listdir(base_folder):
        video_folder_path = os.path.join(base_folder, video_folder)
        if not os.path.isdir(video_folder_path):
            continue

        print(f"Processing video folder: {video_folder}")

        error_level_plot = os.path.join(video_folder_path, 'error_level_plot.png')

        if os.path.exists(error_level_plot):
            features = extract_features(error_level_plot)
            if features is not None and features.size > 0:
                features_output_path = os.path.join(video_folder_path, 'features.npy')
                np.save(features_output_path, features)
                print(f"Features saved for {video_folder}")
            else:
                print(f"No features were extracted for {video_folder}")
        else:
            print(f"Error level plot not found for {video_folder}")

# Example usage
base_folder = "output_frames"
process_video_folders(base_folder)

/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Processing video folder: ahqqqilsxt
Processing image: output_frames/ahqqqilsxt/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features shape: torch.Size([1, 512, 1, 1])
Squeezed features shape: (512,)
Features saved for ahqqqilsxt
Processing video folder: djvtbgwdcc
Processing image: output_frames/djvtbgwdcc/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features shape: torch.Size([1, 512, 1, 1])
Squeezed features shape: (512,)
Features saved for djvtbgwdcc
Processing video folder: atzdznmder
Processing image: output_frames/atzdznmder/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features shape: torch.Size([1, 512, 1, 1])
Squeezed features shape: (512,)
Features saved for atzdznmder
Processing video folder: esyrimvzsa
Processing image: output_frames/esyrimvzsa/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features shape: torch.Size([1, 512, 1, 1])
Squeez

KeyboardInterrupt: 

In [6]:
# ORAGANIZED THE FEATURES.NPY OF EACH VIDEO ACCORDING TO THERE LABEL FROM METADATA.JSON FILE 
import json
import os
import shutil

# Define paths
input_frames_dir = "output_frames"
output_dir = "organized_frames"
metadata_file = "metadata.json"

# Create output directories
os.makedirs(os.path.join(output_dir, "REAL"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "FAKE"), exist_ok=True)

# Read metadata
with open(metadata_file, "r") as f:
    metadata = json.load(f)

# Process each video
for video, info in metadata.items():
    label = info["label"]
    video_name = os.path.splitext(video)[0]  # Remove the .mp4 extension

    # Source and destination paths
    src_path = os.path.join(input_frames_dir, video_name)
    dst_path = os.path.join(output_dir, label, video_name)

    # Check if the source directory exists
    if os.path.exists(src_path):
        # Copy the frame folder to the appropriate category
        shutil.copytree(src_path, dst_path)
        print(f"Copied {video_name} to {label} category")
    else:
        print(f"Warning: Frame folder for {video_name} not found")

print("Organization complete!")

Copied aagfhgtpmv to FAKE category
Copied aapnvogymq to FAKE category
Copied abarnvbtwb to REAL category
Copied abofeumbvv to FAKE category
Copied abqwwspghj to FAKE category
Copied acifjvzvpm to FAKE category
Copied acqfdwsrhi to FAKE category
Copied acxnxvbsxk to FAKE category
Copied acxwigylke to FAKE category
Copied aczrgyricp to FAKE category
Copied adhsbajydo to FAKE category
Copied adohikbdaz to FAKE category
Copied adylbeequz to FAKE category
Copied aelfnikyqj to REAL category
Copied aelzhcnwgf to FAKE category
Copied aettqgevhz to FAKE category
Copied aevrfsexku to FAKE category
Copied afoovlsmtx to REAL category
Copied agdkmztvby to FAKE category
Copied agqphdxmwt to FAKE category
Copied agrmhtjdlk to REAL category
Copied ahbweevwpv to FAKE category
Copied ahdbuwqxit to FAKE category
Copied ahfazfbntc to FAKE category
Copied ahqqqilsxt to REAL category
Copied aipfdnwpoo to FAKE category
Copied ajqslcypsw to REAL category
Copied ajwpjhrbcv to FAKE category
Copied aklqzsddfl to

In [8]:
# DID CLASSIFICATION USING SVM AND KNN
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Define paths
data_dir = "organized_frames"
categories = ["FAKE", "REAL"]

# Initialize data lists
features = []
labels = []

# Load features and labels
for category in categories:
    category_path = os.path.join(data_dir, category, "")
    label = categories.index(category)
    print(f"Processing {category} videos...")

    for video_folder in tqdm(os.listdir(category_path)):
        video_path = os.path.join(category_path, video_folder)
        features_path = os.path.join(video_path, "features.npy")

        if os.path.exists(features_path):
            video_features = np.load(features_path)
            features.append(video_features)
            labels.append(label)
        else:
            print(f"Warning: features.npy not found in {video_path}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Ensure features and labels are not empty
if len(features) == 0:
    raise ValueError("Features array is empty. Please check the data loading process.")

# Flatten the features if necessary
if len(features.shape) > 2:
    features = features.reshape(features.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# SVM Model
print("Training SVM model...")
svm_model = SVC(kernel='sigmoid')  # You can experiment with different kernels like 'rbf'
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Evaluate SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

# KNN Model
print("Training KNN model...")
knn_model = KNeighborsClassifier(n_neighbors=4)  # You can adjust the number of neighbors
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

# Evaluate KNN
print("KNN Classification Report:")
print(classification_report(y_test, y_pred_knn))
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")

Processing FAKE videos...


100%|██████████| 323/323 [00:00<00:00, 5803.81it/s]


Processing REAL videos...


100%|██████████| 77/77 [00:00<00:00, 5472.62it/s]

Training SVM model...
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89        64
           1       0.00      0.00      0.00        16

    accuracy                           0.80        80
   macro avg       0.40      0.50      0.44        80
weighted avg       0.64      0.80      0.71        80

SVM Accuracy: 0.80
Training KNN model...
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.98      0.89        64
           1       0.67      0.12      0.21        16

    accuracy                           0.81        80
   macro avg       0.74      0.55      0.55        80
weighted avg       0.79      0.81      0.76        80

KNN Accuracy: 0.81



/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res

In [10]:
# USE PRETRAINED MODEL INCEPTIONV3 TO EXTRACT FEATURE
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import os

# Load the Inception model
inception_model = models.inception_v3(pretrained=True)

# Switch the model to evaluation mode
inception_model.eval()

# Define the transformation for Inception
inception_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to load and process an image for Inception
def process_image_inception(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        img = inception_transform(img)
        img = img.unsqueeze(0)  # Add batch dimension
        return img
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

# Function to extract features using Inception
def extract_features_inception(image_tensor):
    try:
        if image_tensor is not None and len(image_tensor.shape) == 4:
            with torch.no_grad():
                features = inception_model(image_tensor)
                return features
        else:
            print(f"Error: Invalid input tensor size {image_tensor.shape if image_tensor is not None else None}")
    except Exception as e:
        print(f"Error processing image tensor: {e}")
    return None

# Directory where the images are stored
input_directory = "output_frames_inception"

# Iterate through each video folder
for video_folder in os.listdir(input_directory):
    folder_path = os.path.join(input_directory, video_folder)
    if os.path.isdir(folder_path):
        print(f"Processing video folder: {video_folder}")

        for img_file in os.listdir(folder_path):
            if img_file.endswith('.png'):
                img_path = os.path.join(folder_path, img_file)
                print(f"Processing image: {img_path}")

                # Load and transform image for Inception
                image_tensor = process_image_inception(img_path)

                # Check the shape of the tensor
                if image_tensor is not None:
                    print(f"Image tensor shape: {image_tensor.shape}")

                # Extract features using Inception
                features = extract_features_inception(image_tensor)
                if features is not None:
                    print(f"Extracted features for {img_file}")
                else:
                    print(f"No features were extracted for {img_file}")


Processing video folder: ahqqqilsxt
Processing image: output_frames_inception/ahqqqilsxt/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 299, 299])
Extracted features for error_level_plot.png
Processing video folder: djvtbgwdcc
Processing image: output_frames_inception/djvtbgwdcc/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 299, 299])
Extracted features for error_level_plot.png
Processing video folder: atzdznmder
Processing image: output_frames_inception/atzdznmder/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 299, 299])
Extracted features for error_level_plot.png
Processing video folder: esyrimvzsa
Processing image: output_frames_inception/esyrimvzsa/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 299, 299])
Extracted features for error_level_plot.png
Processing video folder: dptrzdvwpg
Processing image: output_frames_inception/dptrzdvwpg/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 299, 299])
Extracted features for error_

In [11]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import os

# Load the SqueezeNet model
squeezenet_model = models.squeezenet1_1(pretrained=True)

# Switch the model to evaluation mode
squeezenet_model.eval()

# Define the transformation for SqueezeNet
squeezenet_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to load and process an image for SqueezeNet
def process_image_squeezenet(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        img = squeezenet_transform(img)
        img = img.unsqueeze(0)  # Add batch dimension
        return img
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

# Function to extract features using SqueezeNet
def extract_features_squeezenet(image_tensor):
    try:
        if image_tensor is not None and len(image_tensor.shape) == 4:
            with torch.no_grad():
                features = squeezenet_model(image_tensor)
                return features
        else:
            print(f"Error: Invalid input tensor size {image_tensor.shape if image_tensor is not None else None}")
    except Exception as e:
        print(f"Error processing image tensor: {e}")
    return None

# Directory where the images are stored
input_directory = "output_frames_squeezenet"

# Iterate through each video folder
for video_folder in os.listdir(input_directory):
    folder_path = os.path.join(input_directory, video_folder)
    if os.path.isdir(folder_path):
        print(f"Processing video folder: {video_folder}")

        for img_file in os.listdir(folder_path):
            if img_file.endswith('.png'):
                img_path = os.path.join(folder_path, img_file)
                print(f"Processing image: {img_path}")

                # Load and transform image for SqueezeNet
                image_tensor = process_image_squeezenet(img_path)

                # Check the shape of the tensor
                if image_tensor is not None:
                    print(f"Image tensor shape: {image_tensor.shape}")

                # Extract features using SqueezeNet
                features = extract_features_squeezenet(image_tensor)
                if features is not None:
                    print(f"Extracted features for {img_file}")
                else:
                    print(f"No features were extracted for {img_file}")


/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to /Users/aniketsaxena/.cache/torch/hub/checkpoints/squeezenet1_1-b8a52dc0.pth
100%|██████████| 4.73M/4.73M [00:02<00:00, 2.41MB/s]


Processing video folder: ahqqqilsxt
Processing image: output_frames_squeezenet/ahqqqilsxt/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features for error_level_plot.png
Processing video folder: djvtbgwdcc
Processing image: output_frames_squeezenet/djvtbgwdcc/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features for error_level_plot.png
Processing video folder: atzdznmder
Processing image: output_frames_squeezenet/atzdznmder/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features for error_level_plot.png
Processing video folder: esyrimvzsa
Processing image: output_frames_squeezenet/esyrimvzsa/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features for error_level_plot.png
Processing video folder: dptrzdvwpg
Processing image: output_frames_squeezenet/dptrzdvwpg/error_level_plot.png
Image tensor shape: torch.Size([1, 3, 224, 224])
Extracted features for e

In [12]:
#FOR INCEPTION
# ORAGANIZED THE FEATURES.NPY OF EACH VIDEO ACCORDING TO THERE LABEL FROM METADATA.JSON FILE 
import json
import os
import shutil

# Define paths
input_frames_dir = "output_frames_inception"
output_dir = "organized_frames_inception"
metadata_file = "metadata.json"

# Create output directories
os.makedirs(os.path.join(output_dir, "REAL"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "FAKE"), exist_ok=True)

# Read metadata
with open(metadata_file, "r") as f:
    metadata = json.load(f)

# Process each video
for video, info in metadata.items():
    label = info["label"]
    video_name = os.path.splitext(video)[0]  # Remove the .mp4 extension

    # Source and destination paths
    src_path = os.path.join(input_frames_dir, video_name)
    dst_path = os.path.join(output_dir, label, video_name)

    # Check if the source directory exists
    if os.path.exists(src_path):
        # Copy the frame folder to the appropriate category
        shutil.copytree(src_path, dst_path)
        print(f"Copied {video_name} to {label} category")
    else:
        print(f"Warning: Frame folder for {video_name} not found")

print("Organization complete!")

Copied aagfhgtpmv to FAKE category
Copied aapnvogymq to FAKE category
Copied abarnvbtwb to REAL category
Copied abofeumbvv to FAKE category
Copied abqwwspghj to FAKE category
Copied acifjvzvpm to FAKE category
Copied acqfdwsrhi to FAKE category
Copied acxnxvbsxk to FAKE category
Copied acxwigylke to FAKE category
Copied aczrgyricp to FAKE category
Copied adhsbajydo to FAKE category
Copied adohikbdaz to FAKE category
Copied adylbeequz to FAKE category
Copied aelfnikyqj to REAL category
Copied aelzhcnwgf to FAKE category
Copied aettqgevhz to FAKE category
Copied aevrfsexku to FAKE category
Copied afoovlsmtx to REAL category
Copied agdkmztvby to FAKE category
Copied agqphdxmwt to FAKE category
Copied agrmhtjdlk to REAL category
Copied ahbweevwpv to FAKE category
Copied ahdbuwqxit to FAKE category
Copied ahfazfbntc to FAKE category
Copied ahqqqilsxt to REAL category
Copied aipfdnwpoo to FAKE category
Copied ajqslcypsw to REAL category
Copied ajwpjhrbcv to FAKE category
Copied aklqzsddfl to

In [13]:
#FOR SQUEEZENET
# ORAGANIZED THE FEATURES.NPY OF EACH VIDEO ACCORDING TO THERE LABEL FROM METADATA.JSON FILE 
import json
import os
import shutil

# Define paths
input_frames_dir = "output_frames_squeezenet"
output_dir = "organized_frames_squeezenet"
metadata_file = "metadata.json"

# Create output directories
os.makedirs(os.path.join(output_dir, "REAL"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "FAKE"), exist_ok=True)

# Read metadata
with open(metadata_file, "r") as f:
    metadata = json.load(f)

# Process each video
for video, info in metadata.items():
    label = info["label"]
    video_name = os.path.splitext(video)[0]  # Remove the .mp4 extension

    # Source and destination paths
    src_path = os.path.join(input_frames_dir, video_name)
    dst_path = os.path.join(output_dir, label, video_name)

    # Check if the source directory exists
    if os.path.exists(src_path):
        # Copy the frame folder to the appropriate category
        shutil.copytree(src_path, dst_path)
        print(f"Copied {video_name} to {label} category")
    else:
        print(f"Warning: Frame folder for {video_name} not found")

print("Organization complete!")

Copied aagfhgtpmv to FAKE category
Copied aapnvogymq to FAKE category
Copied abarnvbtwb to REAL category
Copied abofeumbvv to FAKE category
Copied abqwwspghj to FAKE category
Copied acifjvzvpm to FAKE category
Copied acqfdwsrhi to FAKE category
Copied acxnxvbsxk to FAKE category
Copied acxwigylke to FAKE category
Copied aczrgyricp to FAKE category
Copied adhsbajydo to FAKE category
Copied adohikbdaz to FAKE category
Copied adylbeequz to FAKE category
Copied aelfnikyqj to REAL category
Copied aelzhcnwgf to FAKE category
Copied aettqgevhz to FAKE category
Copied aevrfsexku to FAKE category
Copied afoovlsmtx to REAL category
Copied agdkmztvby to FAKE category
Copied agqphdxmwt to FAKE category
Copied agrmhtjdlk to REAL category
Copied ahbweevwpv to FAKE category
Copied ahdbuwqxit to FAKE category
Copied ahfazfbntc to FAKE category
Copied ahqqqilsxt to REAL category
Copied aipfdnwpoo to FAKE category
Copied ajqslcypsw to REAL category
Copied ajwpjhrbcv to FAKE category
Copied aklqzsddfl to

In [14]:
# DID CLASSIFICATION USING SVM AND KNN FOR INCEPTION
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Define paths
data_dir = "organized_frames_inception"
categories = ["FAKE", "REAL"]

# Initialize data lists
features = []
labels = []

# Load features and labels
for category in categories:
    category_path = os.path.join(data_dir, category, "")
    label = categories.index(category)
    print(f"Processing {category} videos...")

    for video_folder in tqdm(os.listdir(category_path)):
        video_path = os.path.join(category_path, video_folder)
        features_path = os.path.join(video_path, "features.npy")

        if os.path.exists(features_path):
            video_features = np.load(features_path)
            features.append(video_features)
            labels.append(label)
        else:
            print(f"Warning: features.npy not found in {video_path}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Ensure features and labels are not empty
if len(features) == 0:
    raise ValueError("Features array is empty. Please check the data loading process.")

# Flatten the features if necessary
if len(features.shape) > 2:
    features = features.reshape(features.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# SVM Model
print("Training SVM model...")
svm_model = SVC(kernel='sigmoid')  # You can experiment with different kernels like 'rbf'
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Evaluate SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

# KNN Model
print("Training KNN model...")
knn_model = KNeighborsClassifier(n_neighbors=4)  # You can adjust the number of neighbors
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

# Evaluate KNN
print("KNN Classification Report:")
print(classification_report(y_test, y_pred_knn))
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")

Processing FAKE videos...


100%|██████████| 323/323 [00:00<00:00, 9839.85it/s]


Processing REAL videos...


100%|██████████| 77/77 [00:00<00:00, 8825.05it/s]

Training SVM model...
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89        64
           1       0.00      0.00      0.00        16

    accuracy                           0.80        80
   macro avg       0.40      0.50      0.44        80
weighted avg       0.64      0.80      0.71        80

SVM Accuracy: 0.80
Training KNN model...
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.98      0.89        64
           1       0.67      0.12      0.21        16

    accuracy                           0.81        80
   macro avg       0.74      0.55      0.55        80
weighted avg       0.79      0.81      0.76        80

KNN Accuracy: 0.81



/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res

In [15]:
# DID CLASSIFICATION USING SVM AND KNN FOR SQUEEZENET
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Define paths
data_dir = "organized_frames_squeezenet"
categories = ["FAKE", "REAL"]

# Initialize data lists
features = []
labels = []

# Load features and labels
for category in categories:
    category_path = os.path.join(data_dir, category, "")
    label = categories.index(category)
    print(f"Processing {category} videos...")

    for video_folder in tqdm(os.listdir(category_path)):
        video_path = os.path.join(category_path, video_folder)
        features_path = os.path.join(video_path, "features.npy")

        if os.path.exists(features_path):
            video_features = np.load(features_path)
            features.append(video_features)
            labels.append(label)
        else:
            print(f"Warning: features.npy not found in {video_path}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Ensure features and labels are not empty
if len(features) == 0:
    raise ValueError("Features array is empty. Please check the data loading process.")

# Flatten the features if necessary
if len(features.shape) > 2:
    features = features.reshape(features.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# SVM Model
print("Training SVM model...")
svm_model = SVC(kernel='sigmoid')  # You can experiment with different kernels like 'rbf'
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Evaluate SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

# KNN Model
print("Training KNN model...")
knn_model = KNeighborsClassifier(n_neighbors=4)  # You can adjust the number of neighbors
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

# Evaluate KNN
print("KNN Classification Report:")
print(classification_report(y_test, y_pred_knn))
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")

Processing FAKE videos...


100%|██████████| 323/323 [00:00<00:00, 9949.11it/s]


Processing REAL videos...


100%|██████████| 77/77 [00:00<00:00, 10426.52it/s]

Training SVM model...
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89        64
           1       0.00      0.00      0.00        16

    accuracy                           0.80        80
   macro avg       0.40      0.50      0.44        80
weighted avg       0.64      0.80      0.71        80

SVM Accuracy: 0.80
Training KNN model...
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.98      0.89        64
           1       0.67      0.12      0.21        16

    accuracy                           0.81        80
   macro avg       0.74      0.55      0.55        80
weighted avg       0.79      0.81      0.76        80

KNN Accuracy: 0.81



/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/Implementation/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res

In [19]:
#before code is class imbalanced
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE  # Import SMOTE for oversampling
from tqdm import tqdm

# Define paths
data_dir = "organized_frames_squeezenet"
categories = ["FAKE", "REAL"]

# Initialize data lists
features = []
labels = []

# Load features and labels
for category in categories:
    category_path = os.path.join(data_dir, category, "")
    label = categories.index(category)
    print(f"Processing {category} videos...")

    for video_folder in tqdm(os.listdir(category_path)):
        video_path = os.path.join(category_path, video_folder)
        features_path = os.path.join(video_path, "features.npy")

        if os.path.exists(features_path):
            video_features = np.load(features_path)
            features.append(video_features)
            labels.append(label)
        else:
            print(f"Warning: features.npy not found in {video_path}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Ensure features and labels are not empty
if len(features) == 0:
    raise ValueError("Features array is empty. Please check the data loading process.")

# Flatten the features if necessary
if len(features.shape) > 2:
    features = features.reshape(features.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Apply SMOTE to balance the training data
print("Applying SMOTE to balance classes...")
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# SVM Model with class weights
print("Training SVM model with class weights...")
svm_model = SVC(kernel='sigmoid', class_weight='balanced')  # Use 'balanced' to handle class imbalance
svm_model.fit(X_train_resampled, y_train_resampled)
y_pred_svm = svm_model.predict(X_test)

# Evaluate SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

# KNN Model
print("Training KNN model...")
knn_model = KNeighborsClassifier(n_neighbors=4)  # Adjust the number of neighbors as needed
knn_model.fit(X_train_resampled, y_train_resampled)
y_pred_knn = knn_model.predict(X_test)

# Evaluate KNN
print("KNN Classification Report:")
print(classification_report(y_test, y_pred_knn))
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")


Processing FAKE videos...


100%|██████████| 323/323 [00:00<00:00, 5576.09it/s]


Processing REAL videos...


100%|██████████| 77/77 [00:00<00:00, 6692.81it/s]

Applying SMOTE to balance classes...
Training SVM model with class weights...
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.34      0.48        64
           1       0.21      0.69      0.32        16

    accuracy                           0.41        80
   macro avg       0.51      0.52      0.40        80
weighted avg       0.69      0.41      0.45        80

SVM Accuracy: 0.41
Training KNN model...
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.48      0.61        64
           1       0.23      0.62      0.34        16

    accuracy                           0.51        80
   macro avg       0.54      0.55      0.48        80
weighted avg       0.72      0.51      0.56        80

KNN Accuracy: 0.51


In [22]:
#before code is class imbalanced
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE  # Import SMOTE for oversampling
from tqdm import tqdm

# Define paths
data_dir = "organized_frames_inception"
categories = ["FAKE", "REAL"]

# Initialize data lists
features = []
labels = []

# Load features and labels
for category in categories:
    category_path = os.path.join(data_dir, category, "")
    label = categories.index(category)
    print(f"Processing {category} videos...")

    for video_folder in tqdm(os.listdir(category_path)):
        video_path = os.path.join(category_path, video_folder)
        features_path = os.path.join(video_path, "features.npy")

        if os.path.exists(features_path):
            video_features = np.load(features_path)
            features.append(video_features)
            labels.append(label)
        else:
            print(f"Warning: features.npy not found in {video_path}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Ensure features and labels are not empty
if len(features) == 0:
    raise ValueError("Features array is empty. Please check the data loading process.")

# Flatten the features if necessary
if len(features.shape) > 2:
    features = features.reshape(features.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Apply SMOTE to balance the training data
print("Applying SMOTE to balance classes...")
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# SVM Model with class weights
print("Training SVM model with class weights...")
svm_model = SVC(kernel='sigmoid', class_weight='balanced')  # Use 'balanced' to handle class imbalance
svm_model.fit(X_train_resampled, y_train_resampled)
y_pred_svm = svm_model.predict(X_test)

# Evaluate SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

# KNN Model
print("Training KNN model...")
knn_model = KNeighborsClassifier(n_neighbors=4)  # Adjust the number of neighbors as needed
knn_model.fit(X_train_resampled, y_train_resampled)
y_pred_knn = knn_model.predict(X_test)

# Evaluate KNN
print("KNN Classification Report:")
print(classification_report(y_test, y_pred_knn))
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")


Processing FAKE videos...


100%|██████████| 323/323 [00:00<00:00, 11202.85it/s]


Processing REAL videos...


100%|██████████| 77/77 [00:00<00:00, 11201.88it/s]

Applying SMOTE to balance classes...
Training SVM model with class weights...
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.34      0.48        64
           1       0.21      0.69      0.32        16

    accuracy                           0.41        80
   macro avg       0.51      0.52      0.40        80
weighted avg       0.69      0.41      0.45        80

SVM Accuracy: 0.41
Training KNN model...
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.48      0.61        64
           1       0.23      0.62      0.34        16

    accuracy                           0.51        80
   macro avg       0.54      0.55      0.48        80
weighted avg       0.72      0.51      0.56        80

KNN Accuracy: 0.51


In [20]:
import os
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE  # Import SMOTE for oversampling
from tqdm import tqdm

# Define paths
data_dir = "organized_frames_squeezenet"
categories = ["FAKE", "REAL"]

# Initialize data lists
features = []
labels = []

# Load features and labels
for category in categories:
    category_path = os.path.join(data_dir, category, "")
    label = categories.index(category)
    print(f"Processing {category} videos...")

    for video_folder in tqdm(os.listdir(category_path)):
        video_path = os.path.join(category_path, video_folder)
        features_path = os.path.join(video_path, "features.npy")

        if os.path.exists(features_path):
            video_features = np.load(features_path)
            features.append(video_features)
            labels.append(label)
        else:
            print(f"Warning: features.npy not found in {video_path}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Ensure features and labels are not empty
if len(features) == 0:
    raise ValueError("Features array is empty. Please check the data loading process.")

# Flatten the features if necessary
if len(features.shape) > 2:
    features = features.reshape(features.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Apply SMOTE to balance the training data
print("Applying SMOTE to balance classes...")
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# SVM Model with class weights
print("Training SVM model with class weights...")
svm_model = SVC(kernel='sigmoid', class_weight='balanced')  # Use 'balanced' to handle class imbalance
svm_model.fit(X_train_resampled, y_train_resampled)
y_pred_svm = svm_model.predict(X_test)

# Evaluate SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

# Random Forest with Hyperparameter Tuning
print("Training Random Forest model with hyperparameter tuning...")

# Define the Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)

# Define the hyperparameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Perform GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best estimator from the grid search
best_rf_model = grid_search.best_estimator_
print(f"Best parameters for Random Forest: {grid_search.best_params_}")

# Make predictions with the tuned Random Forest model
y_pred_rf = best_rf_model.predict(X_test)

# Evaluate Random Forest
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.2f}")


Processing FAKE videos...


100%|██████████| 323/323 [00:00<00:00, 6390.44it/s]


Processing REAL videos...


100%|██████████| 77/77 [00:00<00:00, 6093.04it/s]

Applying SMOTE to balance classes...
Training SVM model with class weights...
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.34      0.48        64
           1       0.21      0.69      0.32        16

    accuracy                           0.41        80
   macro avg       0.51      0.52      0.40        80
weighted avg       0.69      0.41      0.45        80

SVM Accuracy: 0.41
Training Random Forest model with hyperparameter tuning...
Fitting 5 folds for each of 216 candidates, totalling 1080 fits


Best parameters for Random Forest: {'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.92      0.85        64
           1       0.00      0.00      0.00        16

    accuracy                           0.74        80
   macro avg       0.39      0.46      0.42        80
weighted avg       0.63      0.74      0.68        80

Random Forest Accuracy: 0.74


In [23]:
import os
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE  # Import SMOTE for oversampling
from tqdm import tqdm

# Define paths
data_dir = "organized_frames_inception"
categories = ["FAKE", "REAL"]

# Initialize data lists
features = []
labels = []

# Load features and labels
for category in categories:
    category_path = os.path.join(data_dir, category, "")
    label = categories.index(category)
    print(f"Processing {category} videos...")

    for video_folder in tqdm(os.listdir(category_path)):
        video_path = os.path.join(category_path, video_folder)
        features_path = os.path.join(video_path, "features.npy")

        if os.path.exists(features_path):
            video_features = np.load(features_path)
            features.append(video_features)
            labels.append(label)
        else:
            print(f"Warning: features.npy not found in {video_path}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Ensure features and labels are not empty
if len(features) == 0:
    raise ValueError("Features array is empty. Please check the data loading process.")

# Flatten the features if necessary
if len(features.shape) > 2:
    features = features.reshape(features.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Apply SMOTE to balance the training data
print("Applying SMOTE to balance classes...")
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# SVM Model with class weights
print("Training SVM model with class weights...")
svm_model = SVC(kernel='sigmoid', class_weight='balanced')  # Use 'balanced' to handle class imbalance
svm_model.fit(X_train_resampled, y_train_resampled)
y_pred_svm = svm_model.predict(X_test)

# Evaluate SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

# Random Forest with Hyperparameter Tuning
print("Training Random Forest model with hyperparameter tuning...")

# Define the Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)

# Define the hyperparameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Perform GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best estimator from the grid search
best_rf_model = grid_search.best_estimator_
print(f"Best parameters for Random Forest: {grid_search.best_params_}")

# Make predictions with the tuned Random Forest model
y_pred_rf = best_rf_model.predict(X_test)

# Evaluate Random Forest
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.2f}")


Processing FAKE videos...


100%|██████████| 323/323 [00:00<00:00, 11449.00it/s]


Processing REAL videos...


100%|██████████| 77/77 [00:00<00:00, 9409.47it/s]

Applying SMOTE to balance classes...
Training SVM model with class weights...
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.34      0.48        64
           1       0.21      0.69      0.32        16

    accuracy                           0.41        80
   macro avg       0.51      0.52      0.40        80
weighted avg       0.69      0.41      0.45        80

SVM Accuracy: 0.41
Training Random Forest model with hyperparameter tuning...
Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.3s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.6s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.3s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=2, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, min_sample

In [24]:
import os
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE  # Import SMOTE for oversampling
from tqdm import tqdm

# Define paths
data_dir = "organized_frames_inception"
categories = ["FAKE", "REAL"]

# Initialize data lists
features = []
labels = []

# Load features and labels
for category in categories:
    category_path = os.path.join(data_dir, category, "")
    label = categories.index(category)
    print(f"Processing {category} videos...")

    for video_folder in tqdm(os.listdir(category_path)):
        video_path = os.path.join(category_path, video_folder)
        features_path = os.path.join(video_path, "features.npy")

        if os.path.exists(features_path):
            video_features = np.load(features_path)
            features.append(video_features)
            labels.append(label)
        else:
            print(f"Warning: features.npy not found in {video_path}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Ensure features and labels are not empty
if len(features) == 0:
    raise ValueError("Features array is empty. Please check the data loading process.")

# Flatten the features if necessary
if len(features.shape) > 2:
    features = features.reshape(features.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Apply SMOTE to balance the training data
print("Applying SMOTE to balance classes...")
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Scale the features for better SVM performance
scaler = StandardScaler()
X_train_resampled = scaler.fit_transform(X_train_resampled)
X_test = scaler.transform(X_test)

# Hyperparameter Tuning for SVM
print("Training SVM model with hyperparameter tuning...")

# Define the SVM classifier
svm_model = SVC()

# Define the hyperparameter grid for GridSearchCV
svm_param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4]
}

# Perform GridSearchCV to find the best hyperparameters for SVM
svm_grid_search = GridSearchCV(estimator=svm_model, param_grid=svm_param_grid, cv=5, n_jobs=-1, verbose=2)
svm_grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best SVM model
best_svm_model = svm_grid_search.best_estimator_
print(f"Best parameters for SVM: {svm_grid_search.best_params_}")

# Make predictions with the tuned SVM model
y_pred_svm = best_svm_model.predict(X_test)

# Evaluate SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

# Random Forest with Hyperparameter Tuning (same as before)
print("Training Random Forest model with hyperparameter tuning...")

# Define the Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)

# Define the hyperparameter grid for GridSearchCV
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Perform GridSearchCV to find the best hyperparameters for Random Forest
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid, cv=5, n_jobs=-1, verbose=2)
rf_grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best Random Forest model
best_rf_model = rf_grid_search.best_estimator_
print(f"Best parameters for Random Forest: {rf_grid_search.best_params_}")

# Make predictions with the tuned Random Forest model
y_pred_rf = best_rf_model.predict(X_test)

# Evaluate Random Forest
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.2f}")


Processing FAKE videos...


100%|██████████| 323/323 [00:00<00:00, 5896.95it/s]


Processing REAL videos...


100%|██████████| 77/77 [00:00<00:00, 6121.79it/s]

Applying SMOTE to balance classes...
Training SVM model with hyperparameter tuning...
Fitting 5 folds for each of 96 candidates, totalling 480 fits


Best parameters for SVM: {'C': 100, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
SVM Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.83      0.80        64
           1       0.00      0.00      0.00        16

    accuracy                           0.66        80
   macro avg       0.38      0.41      0.40        80
weighted avg       0.61      0.66      0.64        80

SVM Accuracy: 0.66
Training Random Forest model with hyperparameter tuning...
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=2, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=2, min

In [ ]:
# SVM O.66 AND KNN 0.74

In [26]:
import os
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE  # Import SMOTE for oversampling
from tqdm import tqdm

# Define paths
data_dir = "organized_frames_squeezenet"
categories = ["FAKE", "REAL"]

# Initialize data lists
features = []
labels = []

# Load features and labels
for category in categories:
    category_path = os.path.join(data_dir, category, "")
    label = categories.index(category)
    print(f"Processing {category} videos...")

    for video_folder in tqdm(os.listdir(category_path)):
        video_path = os.path.join(category_path, video_folder)
        features_path = os.path.join(video_path, "features.npy")

        if os.path.exists(features_path):
            video_features = np.load(features_path)
            features.append(video_features)
            labels.append(label)
        else:
            print(f"Warning: features.npy not found in {video_path}")

# Convert lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Ensure features and labels are not empty
if len(features) == 0:
    raise ValueError("Features array is empty. Please check the data loading process.")

# Flatten the features if necessary
if len(features.shape) > 2:
    features = features.reshape(features.shape[0], -1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Apply SMOTE to balance the training data
print("Applying SMOTE to balance classes...")
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Scale the features for better SVM performance
scaler = StandardScaler()
X_train_resampled = scaler.fit_transform(X_train_resampled)
X_test = scaler.transform(X_test)

# Hyperparameter Tuning for SVM
print("Training SVM model with hyperparameter tuning...")

# Define the SVM classifier
svm_model = SVC()

# Define the hyperparameter grid for GridSearchCV
svm_param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4]
}

# Perform GridSearchCV to find the best hyperparameters for SVM
svm_grid_search = GridSearchCV(estimator=svm_model, param_grid=svm_param_grid, cv=5, n_jobs=-1, verbose=2)
svm_grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best SVM model
best_svm_model = svm_grid_search.best_estimator_
print(f"Best parameters for SVM: {svm_grid_search.best_params_}")

# Make predictions with the tuned SVM model
y_pred_svm = best_svm_model.predict(X_test)

# Evaluate SVM
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_svm))
print(f"SVM Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")

# Random Forest with Hyperparameter Tuning (same as before)
print("Training Random Forest model with hyperparameter tuning...")

# Define the Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)

# Define the hyperparameter grid for GridSearchCV
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Perform GridSearchCV to find the best hyperparameters for Random Forest
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid, cv=5, n_jobs=-1, verbose=2)
rf_grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best Random Forest model
best_rf_model = rf_grid_search.best_estimator_
print(f"Best parameters for Random Forest: {rf_grid_search.best_params_}")

# Make predictions with the tuned Random Forest model
y_pred_rf = best_rf_model.predict(X_test)

# Evaluate Random Forest
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.2f}")


Processing FAKE videos...


100%|██████████| 323/323 [00:00<00:00, 6618.05it/s]


Processing REAL videos...


100%|██████████| 77/77 [00:00<00:00, 6226.12it/s]

Applying SMOTE to balance classes...
Training SVM model with hyperparameter tuning...
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   1.3s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.6s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=50; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=50; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.7s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=50; total time=   0.4s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=50; total time=   0.3s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.6s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2,